In [26]:

import subprocess
import pandas as pd
import io
import numpy as np
import pickle
import os
def get_free_gpu():
    gpu_stats = subprocess.check_output(["nvidia-smi", "--format=csv", "--query-gpu=memory.used,memory.free"])
    gpu_stats = gpu_stats.decode('utf-8')
    gpu_df = pd.read_csv(io.StringIO(gpu_stats))
    gpu_df["memory.free"] = gpu_df[' memory.free [MiB]']
    gpu_df['memory.free'] = gpu_df['memory.free'].map(lambda x: x.rstrip(' [MiB]')).astype('float32')
    idx = gpu_df['memory.free'].idxmax()
    print('Returning GPU{} with {} free MiB'.format(idx, gpu_df.iloc[idx]['memory.free']))
    return idx

# Hey Idiot before going down get the fucking gpu first

## This works for LM WaterMarking and Reliability of AI Text detector

In [27]:

import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from LMwatermarking.watermark_processor import  WatermarkDetector


def detect(input_text,device=None, tokenizer=None):
    watermark_detector = WatermarkDetector(vocab=list(tokenizer.get_vocab().values()),
                                        gamma=0.25,
                                        seeding_scheme='simple_1',
                                        device=device,
                                        tokenizer=tokenizer,
                                        z_threshold=4.0,
                                        normalizers=[],
                                        select_green_tokens=True)
    if len(input_text)-1 > watermark_detector.min_prefix_len:
        score_dict = watermark_detector.detect(input_text)
        return score_dict


def detect_watermark(document):
    device = "cuda:" + str(get_free_gpu()) if torch.cuda.is_available() else "cpu"   
    tokenizer = LlamaTokenizer.from_pretrained("./llama-2-7b-hf" )
    torch.manual_seed(0)
    scores = []
    for text in document:
        score  = detect(text , device , tokenizer)
        # print(score)
        scores.append(score['z_score'])
    return scores
        
   

In [33]:
import glob
import numpy as np
def zscore_to_percentile(z):
    return (1 - np.exp(-0.5 * z**2 / np.pi)) * 100
for files in glob.glob('Dataset/Attacked/NewData/*/*/*'):
    if('sir' in files.lower() or 'kwg' in files.lower()):
        print(files)
        with open(files, 'rb') as f:
            data = pickle.load(f)
        scores = detect_watermark(data)
        
        vec_zscore_to_percentile = np.vectorize(zscore_to_percentile)

        scores = np.array(scores)
        scores = vec_zscore_to_percentile(scores)
        
        
        if(os.path.exists("/".join(files.split('/')[:-1]).replace('NewData','NewDataScores'))):
            pass
        else:
            os.makedirs("/".join(files.split('/')[:-1]).replace('NewData','NewDataScores'))
        
        with open(files.replace('NewData','NewDataScores').replace('.pkl','_scores.pkl'), 'wb') as f:
            pickle.dump(scores, f)
    

Dataset/Attacked/NewData/Paraphraed_PivotTranslation/sir/llm_watermarked_sir_pivot_translated.pkl
Returning GPU7 with 27941.0 free MiB


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Dataset/Attacked/NewData/Paraphraed_PivotTranslation/kwg/llm_watermarked_kwg_pivot_translated.pkl
Returning GPU7 with 27941.0 free MiB


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Dataset/Attacked/NewData/Paraphrased_NormalTranslation/sir/llm_watermarked_sir_translated.pkl
Returning GPU7 with 27941.0 free MiB


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Dataset/Attacked/NewData/translation_paraphrased/kwg/rephrased3.pkl
Returning GPU7 with 27941.0 free MiB


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Dataset/Attacked/NewData/translation_paraphrased/kwg/rephrased1.pkl
Returning GPU7 with 27941.0 free MiB


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Now this is for robust waterMarking scheme


In [24]:
import json
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import LogitsProcessorList
from RobustWatermark.watermark import WatermarkLogitsProcessor, WatermarkWindow, WatermarkContext
import argparse
import os
from transformers import LlamaTokenizer ,  LlamaTokenizer, AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import pickle

    
def detect_watermark(args ):
    import subprocess
    import sys
    import pandas as pd
    import io

    def get_free_gpu():
        gpu_stats = subprocess.check_output(["nvidia-smi", "--format=csv", "--query-gpu=memory.used,memory.free"])
        gpu_stats = gpu_stats.decode('utf-8')
        gpu_df = pd.read_csv(io.StringIO(gpu_stats))
        gpu_df["memory.free"] = gpu_df[' memory.free [MiB]']
        gpu_df['memory.free'] = gpu_df['memory.free'].map(lambda x: x.rstrip(' [MiB]')).astype('float32')
        idx = gpu_df['memory.free'].idxmax()
        print('Returning GPU{} with {} free MiB'.format(idx, gpu_df.iloc[idx]['memory.free']))
        return idx
    device = torch.device("cuda:" + str(get_free_gpu()) if torch.cuda.is_available() else "cpu")

    model_path = args.llm_path
    
   # model = AutoModelForCausalLM.from_pretrained(model_path).to(device)
    tokenizer = LlamaTokenizer.from_pretrained(model_path)
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token
       # model.config.pad_token_id = model.config.eos_token_id

    if args.watermark_type == "window": # use a window of previous tokens to hash, e.g. KGW
        watermark_model = WatermarkWindow(device, args.window_size, tokenizer)
        logits_processor = WatermarkLogitsProcessor(watermark_model)
        
        
    elif args.watermark_type == "context":
        print("Hello")
        watermark_model = WatermarkContext(device, args.chunk_size, tokenizer, delta = args.delta,transform_model_path=args.transform_model, embedding_model=args.embedding_model)
        logits_processor = WatermarkLogitsProcessor(watermark_model)
    else:
        watermark_model, logits_processor = None, None

    with torch.no_grad():
        z_score_generated = watermark_model.detect("To calculate true positives, true negatives, false positives, and false negatives, you can use a confusion matrix.") if watermark_model else 0
       # z_score_origin = watermark_model.detect(original_text_file) if watermark_model else 0
       # z_score_attacked = watermark_model.detect(attacked_text) if watermark_model else 0
        print(z_score_generated)


from collections import namedtuple

args_dict = {
    'watermark_type': 'context',
    'base_model': 'llama',
    'llm_path': './llama-2-7b-hf',
    'window_size': 0,
    'generate_number': 200,
    'delta': 1.0,
    'chunk_size': 10,
    'max_new_tokens': 50,
    'data_path': 'data/dataset/prompt.pkl',
    'output_path': 'SIR.json',
    'transform_model': './RobustWatermark/model/transform_model_cbert.pth',
    'embedding_model': './RobustWatermark/data/compositional-bert-large-uncased',
    'decode_method': 'sample',
    'prompt_size': 30,
    'beam_size': 5
}

Args = namedtuple('Args', args_dict.keys())
args = Args(*args_dict.values())


detect_watermark(args)



Returning GPU7 with 29265.0 free MiB
Hello


Some weights of BertModel were not initialized from the model checkpoint at ./RobustWatermark/data/compositional-bert-large-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.819 seconds.
Prefix dict has been built successfully.


Tocalculatetruepositives,truenegatives,falsepositives ['Tocalculatetruepositives', ',', 'truenegatives', ',', 'falsepositives']
Tocalculatetruepositives,truenegatives,falsepositives,andfalsenegatives,youcanuseaconfusion ['Tocalculatetruepositives', ',', 'truenegatives', ',', 'falsepositives', ',', 'andfalsenegatives', ',', 'youcanuseaconfusion']
0.09556977450847626


In [25]:
def main(args):

    device = torch.device("cuda:" + str(get_free_gpu()) if torch.cuda.is_available() else "cpu")

    model_path = args.llm_path
    model = AutoModelForCausalLM.from_pretrained(model_path).to(device)
    tokenizer = LlamaTokenizer.from_pretrained(model_path)
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id

    if args.watermark_type == "window": # use a window of previous tokens to hash, e.g. KGW
        watermark_model = WatermarkWindow(device, args.window_size, tokenizer)
        logits_processor = WatermarkLogitsProcessor(watermark_model)
    elif args.watermark_type == "context":
        watermark_model = WatermarkContext(device, args.chunk_size, tokenizer, delta = args.delta,transform_model_path=args.transform_model, embedding_model=args.embedding_model)
        logits_processor = WatermarkLogitsProcessor(watermark_model)
    else:
        watermark_model, logits_processor = None, None
   
        text = "How are you you"
        words = text.split()
        words = words[:args.prompt_size]
        begin_text = ' '.join(words)
        inputs = tokenizer(begin_text, return_tensors="pt").to(device)
        
        generation_config = {
                "max_length": args.max_new_tokens ,
                "no_repeat_ngram_size": 4,
            }
        if args.decode_method == "sample":
            generation_config["do_sample"] = True
        elif args.decode_method == "beam":
            generation_config["num_beams"] = args.beam_size
            generation_config["do_sample"] = False
        
        if watermark_model is not None:
            generation_config["logits_processor"] = LogitsProcessorList([logits_processor])
        

        with torch.no_grad():
            outputs = model.generate(**inputs, **generation_config)
            print(outputs)


from collections import namedtuple

args_dict = {
    'watermark_type': 'context',
    'base_model': 'llama',
    'llm_path': './llama-2-7b-hf',
    'window_size': 0,
    'generate_number': 200,
    'delta': 1.0,
    'chunk_size': 10,
    'max_new_tokens': 50,
    'data_path': 'data/dataset/prompt.pkl',
    'output_path': 'SIR.json',
    'transform_model': './RobustWatermark/model/transform_model_cbert.pth',
    'embedding_model': './RobustWatermark/data/compositional-bert-large-uncased',
    'decode_method': 'sample',
    'prompt_size': 30,
    'beam_size': 5
}

Args = namedtuple('Args', args_dict.keys())
args = Args(*args_dict.values())


detect_watermark(args)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Returning GPU7 with 27941.0 free MiB
Hello


Some weights of BertModel were not initialized from the model checkpoint at ./RobustWatermark/data/compositional-bert-large-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tocalculatetruepositives,truenegatives,falsepositives ['Tocalculatetruepositives', ',', 'truenegatives', ',', 'falsepositives']
Tocalculatetruepositives,truenegatives,falsepositives,andfalsenegatives,youcanuseaconfusion ['Tocalculatetruepositives', ',', 'truenegatives', ',', 'falsepositives', ',', 'andfalsenegatives', ',', 'youcanuseaconfusion']
0.09556977450847626
